In [1]:
import gensim
from collections import defaultdict
from scipy import spatial
from gensim.models import KeyedVectors
from igraph import *
import random
import pandas as pd
from collections import Counter
import glob
import pickle

In [8]:
def train_word2vec(documents, dim=300, min_count=2, iters=100, window=5, negative=5):
	model = gensim.models.Word2Vec(
        documents,
        sg=1,
        size=dim,
        window=window,
        min_count=min_count,
#         max_final_vocab=3000,
        sample=1e-5,
        iter=iters,
        ns_exponent=0.75,
        negative=negative,
        workers=4)
	model.train(documents, total_examples=len(documents), epochs=model.epochs)
	return model


In [2]:
def read_docs(csv_file, column='stem'):
    """read stem utterances from childes csv files"""
    df = pd.read_csv(csv_file)
    tags = df['part_of_speech'].values
    stems = df['stem'].values
    ret_list = []
    for t, s in zip(tags, stems):
        tl, sl = str(t).lower().split(), str(s).lower().split()
        
        # replace NAME and interjections with $name$ and $co$ respectively
        ntl = []
        for t, s in zip(tl, sl):
            if t == "n:prop":
                ntl.append('$name$')
#             elif t == 'co':
#                 ntl.append('$co$')
            else:
                ntl.append(s)

#         print(' '.join(ntl))
        ret_list.append(ntl)
    return ret_list

In [3]:
childes_files = sorted(glob.glob("./data/childes/period*.csv"))
num_tokens = []
for filename in sorted(childes_files, key=lambda x: int(x.split('_')[-1][:-4])):
    month = int(filename.split('_')[-1][:-4])
    lines = read_docs(filename)
    num_tokens.append(sum([len(l) for l in lines]))
print(num_tokens)

/Users/hang/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/Users/hang/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


[0, 0, 0, 675, 0, 2996, 15663, 17582, 25704, 43371, 2235, 12517, 59629, 26447, 87418, 41542, 64743, 73700, 88824, 129136, 141505, 137879, 126901, 123895, 119761, 167198, 154189, 156461, 186937, 234921, 219216, 160672, 149239, 132535, 134087, 104115, 144722, 103112, 89463, 107917, 91562, 63383, 55278, 73375, 73777, 96541, 74902, 60297, 101080, 51513, 61349, 58460, 39978, 37533, 52838, 45696, 36225, 611807, 33982, 47450, 32701, 19611, 21370, 22121, 30397, 17250, 31727, 42173, 26308, 30721, 22251, 18514, 8091, 6720, 2949, 593, 5993, 1607, 3155, 2907, 989, 1520, 1395, 7262, 7126, 16220, 10471, 9172, 18558, 14202, 15056, 19768, 9056, 1111, 793, 983, 313, 0, 1619, 565, 1677, 2296, 140, 933, 214, 121, 4442, 1450, 104, 7237, 4167, 5642, 9115, 3481, 9926, 6306, 12295, 11186, 1865, 2789, 8263, 5999, 2653, 4866, 0, 1711, 2579, 0, 0, 0, 477, 0, 2484, 1491, 0, 0, 0, 140, 0, 0, 845, 681, 0, 0, 667, 0, 0, 0, 0, 0, 0]


In [7]:
window = []
months = []
periods = []

for i, num in enumerate(num_tokens):
    window.append(num)
    months.append(i)
    if sum(window) > 1000000:
        periods.append(months)
        print(sum(window), months, sum(months)/len(months))
        window = []
        months = []

1098467 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22] 11.0
1143362 [23, 24, 25, 26, 27, 28, 29] 26.0
1044586 [30, 31, 32, 33, 34, 35, 36] 33.0
1042200 [37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49] 43.0
1025318 [50, 51, 52, 53, 54, 55, 56, 57, 58, 59] 54.5


In [11]:
import numpy as np
childes_files = sorted(glob.glob("./data/childes/period*.csv"), key=lambda x: int(x.split('_')[-1][:-4]))
year2vecs = {}
for i, period in enumerate(periods):
    print(np.array(childes_files)[period])
    print('key', 'period'+str(i))
    docs = []
    for filename in np.array(childes_files)[period]:
        docs.extend(read_docs(filename))
    model = train_word2vec(docs, dim=100, min_count=15, iters=50, window=5)
    d = {w:v for w, v in zip(model.wv.index2word, model.wv.vectors)}
    year2vecs['period'+str(i)] = d
print(num_tokens)

['./data/childes/period_0.csv' './data/childes/period_1.csv'
 './data/childes/period_2.csv' './data/childes/period_3.csv'
 './data/childes/period_4.csv' './data/childes/period_5.csv'
 './data/childes/period_6.csv' './data/childes/period_7.csv'
 './data/childes/period_8.csv' './data/childes/period_9.csv'
 './data/childes/period_10.csv' './data/childes/period_11.csv'
 './data/childes/period_12.csv' './data/childes/period_13.csv'
 './data/childes/period_14.csv' './data/childes/period_15.csv'
 './data/childes/period_16.csv' './data/childes/period_17.csv'
 './data/childes/period_18.csv' './data/childes/period_19.csv'
 './data/childes/period_20.csv' './data/childes/period_21.csv'
 './data/childes/period_22.csv']
key period0
['./data/childes/period_23.csv' './data/childes/period_24.csv'
 './data/childes/period_25.csv' './data/childes/period_26.csv'
 './data/childes/period_27.csv' './data/childes/period_28.csv'
 './data/childes/period_29.csv']
key period1
['./data/childes/period_30.csv' './dat

In [12]:
with open('./data/embeddings-over-time/embeddings-English-NA-1M-ep50-f15.pickle', 'wb') as handle:
    pickle.dump(year2vecs, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)